In [85]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [86]:
import torch
import torch.optim as optim 
from utils.auxiliary_fn import moco_train_one_epoch,moco_evaluate_one_epoch,MOCO,MomentumEncoder,InfonceLoss

from utils.loss_fn.standard_loss import mase_loss,huber_loss,log_cosh_loss,pinball_loss,wmape_loss,smape_loss

# split data

In [87]:
from utils.train_val_test import moco_h_d_train,moco_h_d_val,moco_h_d_test,get_h_f_dataloader

span_len=120
pre_len=1
batch_size=8

train_dataset=moco_h_d_train(cut_len=span_len,slide_win_size=pre_len)
val_dataset=moco_h_d_val(cut_len=span_len,slide_win_size=pre_len)
test_dataset=moco_h_d_test(cut_len=span_len,slide_win_size=pre_len)


train_dataloader=get_h_f_dataloader(train_dataset,batch_size,0)
val_dataloader=get_h_f_dataloader(val_dataset,batch_size,0)
test_dataloader=get_h_f_dataloader(test_dataset,batch_size,0)

print(len(train_dataloader))

x1,x2,y=next(iter(train_dataloader))
# print(x)
print(x1.shape)
print(x2.shape)
print(y.shape)




168
torch.Size([8, 120, 2])
torch.Size([8, 120, 2])
torch.Size([8])


d:\kobe\lib\site-packages\sklearn\preprocessing\_data.py:2583: UserWarning: n_quantiles (1000) is greater than the total number of samples (182). n_quantiles is set to n_samples.
  warnings.warn(
d:\kobe\lib\site-packages\sklearn\preprocessing\_data.py:2583: UserWarning: n_quantiles (1000) is greater than the total number of samples (182). n_quantiles is set to n_samples.
  warnings.warn(
d:\kobe\lib\site-packages\sklearn\preprocessing\_data.py:2583: UserWarning: n_quantiles (1000) is greater than the total number of samples (182). n_quantiles is set to n_samples.
  warnings.warn(
d:\kobe\lib\site-packages\sklearn\preprocessing\_data.py:2583: UserWarning: n_quantiles (1000) is greater than the total number of samples (182). n_quantiles is set to n_samples.
  warnings.warn(


# try transformer

In [88]:
from models.LSTM_FCN import MOCO_LSTM_FCN_2_Atten
from utils.loss_fn.soft_dtw import SoftDTW

hidden_dim=64
main_encoder=MOCO_LSTM_FCN_2_Atten(
    data_len=span_len,
    pre_len=pre_len,
    input_dim=2,
    hidden_dim=hidden_dim,
    num_layers=4)

momentum_encoder=MomentumEncoder(main_encoder)

model=MOCO(
    seq_len=span_len,
    feature_dim=hidden_dim,
    pre_len=pre_len,
    main_model=main_encoder,
    moco_model=momentum_encoder,
    pred_los=mase_loss,
    contras_los=InfonceLoss,
    temperature=0.08,
    lamba_1=0.2,
    lambda_2=0.8,
    )

learning_rate=0.015
# optimizer = optim.Adam(model.parameters(),lr=learning_rate,betas=(0.9,0.999))
optimizer=optim.SGD(model.parameters(),lr=learning_rate)

# Define the loss function
# loss_fn = nn.SmoothL1Loss()  ## optional_1

# from utils.loss_fn.soft_dtw import SoftDTW
# # Create the "criterion" object
# loss_fn = SoftDTW(use_cuda=False, gamma=0.1)


# Create the data loader
if torch.cuda.is_available():
    device=torch.device('cuda')
else :
    device=torch.device('cpu')
# Train the model

num_epochs = 1000


In [89]:
def train_and_evaluate(model, optimizer, train_loader, evaluate_loader,val_loader,train_fn,test_fn,device,num_epochs):
    """
    Trains and evaluates a PyTorch model for time series forecasting.
    
    Args:
        model: The PyTorch model to train and evaluate.
        criterion: The loss function to use for training and evaluation.
        optimizer: The optimizer to use for training.
        train_loader (torch.utils.data.DataLoader): The data loader to use for training.
        val_loader (torch.utils.data.DataLoader): The data loader to use for evaluation.
        num_epochs (int): The number of epochs to train the model for.
    """
    avg_train_loss_idx,val_loss_idx=100000,100000
    for epoch in range(num_epochs):
        avg_train_loss=train_fn(model, train_loader, optimizer, device, epoch)
        if avg_train_loss<avg_train_loss_idx:
            avg_train_loss_idx=avg_train_loss
            print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}")
        
        # Evaluation loop
        model.eval()
        val_loss = test_fn(model, val_loader, device, epoch)
        if val_loss<val_loss_idx:
            val_loss_idx=val_loss
            print(f"Epoch [{epoch+1}/{num_epochs}], Val Loss: {val_loss:.4f}")
    
    # Final evaluation on validation set
    model.eval()
    val_loss = test_fn(model, evaluate_loader, device, epoch)
    print(f"Final Val Loss: {val_loss:.4f}")

In [90]:
train_and_evaluate(
    model=model,
    optimizer=optimizer,
    train_loader=train_dataloader,
    val_loader=val_dataloader,
    evaluate_loader=test_dataloader,
    train_fn=moco_train_one_epoch,
    test_fn=moco_evaluate_one_epoch,
    device=device,
    num_epochs=num_epochs)

Epoch [1/1000], Train Loss: 8.9495
Epoch [1/1000], Val Loss: 8.4950
